# Dev mp-server (esp32, server)

## Micropython server

In [1]:
%discover
%connect test-stm32
%softreset

import start_urpc
from urpc import *

import sys
sys_ = import_('sys')

print("urpc version", version_())

print("Platforms:")
print("   host:    {}".format(sys.platform))
print("   server:  {}".format(sys_.get_('platform')))

import network

wlan = network.WLAN(network.STA_IF)
print(wlan.ifconfig()[0])

test-stm32                     serial:///dev/cu.usbmodem208E307542521  
test-esp32                     serial:///dev/cu.usbserial-0160B5B8     
test-stm32-cop                 serial:///dev/cu.usbserial-0143520E     
test-argon                     serial:///dev/cu.usbmodem1413101        
test-samd                      serial:///dev/cu.usbmodem1413201        
Connected to test-stm32 @ serial:///dev/cu.usbmodem208E307542521

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!   softreset ...     !!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

urpc version V1
Platforms:
   host:    pyboard
   server:  esp32
10.39.40.129


In [1]:
import os, io, sys, socket, struct, msgpack, secrets

try:
    from iot import dupterm
except ImportError:
    from uos import dupterm

class Server(io.IOBase):
    
    def __init__(self, ss):
        self.ss = ss
        self.cs = None
            
    def write(self, data):
        if self.cs:
            try:
                self.print("write", data)
                self.cs.write(data)
            except Exception as e:
                self.print("*** write", e)
        else:
            self.print("write", data)
        return len(data)

    def readinto(self, b):
        dupterm(None)
        print("????", b)
        if not self.cs:
            print("accept ...")
            try:
                self.cs, addr = self.ss.accept()
            except Exception as e:
                self.print("*** accept", e)
            self.print("connected from", addr)
        try:
            x = self.cs.read(1)
        except Exception as e:
            self.print("*** read", e)
        if x == b'':
            return 0
        b[0:1] = x
        return 1
    
    def print(self, *args):
        return
        d = dupterm(None)
        print("D", d)
        print(*args)
        dupterm(d)
        
        
def serve(port=1234):
    global ss, server
    ss = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    ss.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    ss.setblocking(True)

    ai = socket.getaddrinfo('', port)
    addr = ai[0][4]
    ss.bind(addr)
    ss.listen(1)

    print("waiting for connection ...")
    server = Server(ss)
    server.print("dupterm ...")
    dterm = dupterm(server)



def start_server(port=54321):

    # advertise
    import network, time
    # network.WLAN(network.STA_IF).mdns_add_service('_mp', '_tcp', port)

    ip = network.WLAN(network.STA_IF).ifconfig()[0]
    print("mp://{}:{}".format(ip, port))

    serve(port)

start_server()

server.print("DONE")

mp://10.39.40.129:54321
waiting for connection ...


In [1]:
print("X")
dupterm(None)
print("ss", ss)
ss.close()

X
ss <socket>
